In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pickle

In [3]:
df = pd.read_csv("D:\Study\Projects\Coffee Roast\coffee_data.csv")

In [4]:
df.head()

,temperature,time,target
0,185.317638,12.693965,1.0
1,259.920475,11.867664,0.0
2,231.013571,14.414242,0.0
3,175.366645,11.720587,0.0
4,187.120865,14.129732,1.0


In [5]:
X = df.loc[:,df.columns != "target"]
y = df.target

print(X.shape, y.shape)

(200, 2) (200,)


In [19]:
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)
Xn = norm_l(X)

In [20]:
Xt = np.tile(Xn,(1000,1))
Yt= np.tile(y,(1000))   
print(Xt.shape, Yt.shape)

(200000, 2) (200000,)


In [21]:
model = Sequential(
    [
        tf.keras.Input(shape=(2,)),
        Dense(3, activation='sigmoid', name = 'layer1'),
        Dense(1, activation='sigmoid', name = 'layer2')
     ]
)

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [23]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"W2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)

W1(2, 3):
 [[ 0.5472162   0.5606847  -0.52198386]
 [ 0.5686053   0.6914824  -0.0478934 ]] 
b1(3,): [0. 0. 0.]
W2(3, 1):
 [[-1.216104  ]
 [ 1.1836847 ]
 [ 0.12708712]] 
b2(1,): [0.]


In [24]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    Xt,Yt,            
    epochs=10,
)

Epoch 1/10
6250/6250 [==============================] - 5s 769us/step - loss: 0.1858
Epoch 2/10
6250/6250 [==============================] - 5s 777us/step - loss: 0.1025
Epoch 3/10
6250/6250 [==============================] - 5s 762us/step - loss: 0.0242
Epoch 4/10
6250/6250 [==============================] - 5s 773us/step - loss: 0.0137
Epoch 5/10
6250/6250 [==============================] - 5s 765us/step - loss: 0.0093
Epoch 6/10
6250/6250 [==============================] - 5s 746us/step - loss: 0.0067
Epoch 7/10
6250/6250 [==============================] - 5s 760us/step - loss: 0.0049
Epoch 8/10
6250/6250 [==============================] - 5s 751us/step - loss: 0.0036
Epoch 9/10
6250/6250 [==============================] - 5s 791us/step - loss: 0.0027
Epoch 10/10
6250/6250 [==============================] - 5s 782us/step - loss: 0.0021


In [25]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[13.638767   14.364012   -0.07102504]
 [11.360463    0.36953872 -9.163034  ]] 
b1: [  1.6850811  15.091257  -11.399081 ]
W2:
 [[-46.01228 ]
 [ 39.472183]
 [-46.900784]] 
b2: [-12.091455]


In [26]:
inputs = tf.keras.Input(shape=(2,))
x = norm_l(inputs)
outputs = model(x)
inference_model = tf.keras.Model(inputs, outputs)

In [27]:
X_input = np.array([200,13.9]).reshape(1,-1)
prediction = inference_model.predict(X_input)
yhat = (prediction >= 0.5).astype(int)

1/1 [==============================] - 0s 90ms/step


In [28]:
print(prediction)
print(yhat)

[[0.9805798]]
[[1]]


In [30]:
pickle.dump(inference_model, open("coffee_model.pkl", "wb"))

INFO:tensorflow:Assets written to: ram://3be0468f-58f3-4560-a40b-618c51571fff/assets
